In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
datagen = ImageDataGenerator(rescale=1/255)
train_gen = datagen.flow_from_directory('./Downloads/MaskClassification',
                                      target_size=(200,200),
                                      batch_size = 128,
                                      class_mode='binary'
                                     )
val_gen = datagen.flow_from_directory('./Downloads/archive123',
                                      target_size=(200,200),
                                      batch_size = 128,
                                      class_mode='binary'
                                     )

Found 10000 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [3]:
model = Sequential()


model.add(Conv2D(32,(3,3),input_shape=(200,200,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers




model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(128,activation='relu'))
#Dense layer of 128 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

In [4]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [5]:
model.fit_generator(train_gen, 
                    steps_per_epoch=int(train_gen.n/128),
                    validation_data=val_gen,
                    validation_steps=int(val_gen.n) // 128,
                    epochs = 3)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
78/78 [==============================] - 260s 3s/step - loss: 0.4527 - accuracy: 0.9076 - val_loss: 0.0740 - val_accuracy: 0.9810
Epoch 2/3
78/78 [==============================] - 264s 3s/step - loss: 0.0452 - accuracy: 0.9844 - val_loss: 0.0495 - val_accuracy: 0.9833
Epoch 3/3
78/78 [==============================] - 278s 4s/step - loss: 0.0298 - accuracy: 0.9885 - val_loss: 0.0261 - val_accuracy: 0.9900


In [149]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
test_image = image.load_img('./Downloads/me2.jpeg', target_size = (200,200,3))
test_image = image.img_to_array(test_image)
test_image /= 255
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
answer = model.predict_classes(test_image)
train_gen.class_indices
print(answer)
if answer[0]==0:
    prediction = 'mask'
else :
    prediction = 'unmask'
print(prediction)

[0]
mask


In [151]:
model_json = model.to_json()
with open("mask-detection-model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("mask-detection-model.h5")
print("Saved model to disk")

Saved model to disk
